In [1]:
import pyreadr, pickle
import numpy as np
import pandas as pd


import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [2]:
N = 10 # How many genes to extract


path = '/home/sam/Classes/Stats/Bayes/assignments/keyGeneExpressionBySubtype.RData'
path = '/home/sam/scRNAseq/RNAscope/RNAscope/RGC_df354.Rdata'
rdata = pyreadr.read_r(path) # WS
# rdata = pyreadr.read_r('/home/sam/Downloads/keyGeneExpressionBySubtype.RData') #Laptop
# genes = rdata['key_genes']

In [3]:
# Load data
df = rdata['df']
df['Cluster'] = df['Cluster'].apply(lambda x: x if len(x.split('_')[0]) == 2 else '0' + x) # Standardize cluster names

# Encode the categoric response 
le = LabelEncoder()
df['Cluster'] = le.fit_transform(df['Cluster'])

# Move the response to the end for simply manipulation
cluster_col = df.pop('Cluster')
df.insert(len(df.columns), 'Cluster', cluster_col)

# Remove RBPMS negative cells
df = df[df['Rbpms'] != 0]
df.drop('Rbpms', axis=1, inplace=True)

# Shuffle the data
df = shuffle(df, random_state=42)

# Split the data into input features and labels
X = df.iloc[:, :-1].values.astype(np.float32)
y = df.iloc[:, -1].values.astype(np.compat.long)

# Convert data to PyTorch tensors
X = torch.from_numpy(X)
y = torch.from_numpy(y)

# Split the data into training and test sets
train_size = int(0.8 * len(df))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

/tmp/ipykernel_121607/3880050509.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('Rbpms', axis=1, inplace=True)


In [4]:
# Define the neural network architecture
# class Net(nn.Module):
#     def __init__(self, input_size, num_classes,
#                  h1_size, h2_size, h3_size, dropout_prob):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(input_size, h1_size)
#         self.fc2 = nn.Linear(h1_size, h2_size)
#         self.fc3 = nn.Linear(h2_size, h3_size)
#         self.fc4 = nn.Linear(h3_size, num_classes)
#         self.dropout = nn.Dropout(p=dropout_prob)

#     def forward(self, x):
#         x = nn.functional.relu(self.fc1(x))
#         x = nn.functional.relu(self.fc2(x))
#         x = self.dropout(x)
#         x = nn.functional.relu(self.fc3(x))
#         x = self.dropout(x)
#         x = self.fc4(x)
#         return nn.functional.log_softmax(x, dim=1)


class Net(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        # self.fc2 = nn.Linear(h1_size, h2_size)
        # self.fc3 = nn.Linear(h2_size, h3_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        # x = nn.functional.relu(self.fc2(x))
        # x = self.dropout(x)
        # x = nn.functional.relu(self.fc3(x))
        # x = self.dropout(x)
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)
    


In [5]:
# This cell is an experiment in training currciulumn
# The idea is to specifically train networks to learn one class only
# To do this we will train it head on against one other class ata a time
# When trainign is complete the network will have been exposed to the desired class at least 44 times
# It will see all non-desired classes at least once
# In this instantiation the network will not be weighted ahead of time to know what class is the desired class
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Step 1: Prepare your training data
def head2head(train_y, train_X, desired, noise):
    '''This function will take a desired category and a noise category and create a paired down dataset
    The paired down data set will be an equal mix of desired and noise
    If there are not enough noise examples, a random set will be duplicated to make the lengths match
    If there are too many noise samples, a random set will be excluded'''

    # Isolate the desired instances
    TS_desired_y = train_y[train_y == desired]
    TS_desired_X = train_X[train_y == desired]
    TS_noise_y = train_y[train_y == noise]
    TS_noise_X = train_X[train_y == noise]
    # Check if the desired class is longer, if so balance it with replicates from te noise
    if len(TS_desired_y) > len(TS_noise_y):
        delta = len(TS_desired_y) - len(TS_noise_y)
        indice = np.random.choice( range(len(TS_noise_y)), delta)
        indice = torch.tensor(indice)
        extra_y = TS_noise_y[indice]
        extra_X = TS_noise_X[indice]
        TS_noise_y = torch.cat((TS_noise_y,extra_y))
        TS_noise_X = torch.cat((TS_noise_X,extra_X))
    # Check if the desired class is shorter, if so randomely discard examples from the other
    elif len(TS_desired_y) < len(TS_noise_y):
        indice = np.random.choice( range(len(TS_noise_y)), len(TS_desired_y))
        indice = torch.tensor(indice)
        TS_noise_y = TS_noise_y[indice]
        TS_noise_X = TS_noise_X[indice]
    # Combine the two balanced data sets
    TS_y = torch.cat((TS_noise_y,TS_desired_y))
    TS_X = torch.cat((TS_noise_X,TS_desired_X))
    # Shuffle the data
    indice = np.random.choice( range(len(TS_y)), len(TS_y))
    indice = torch.tensor(indice)
    TS_y = TS_y[indice]
    TS_X = TS_X[indice]
    return TS_y, TS_X


def DesiredGeneCurriculum(train_y, train_X, desired=1):
    if desired!=None:
        # Create a training set using this function to create my desired class biased training set
        noise_inds = [i for i in np.unique(train_y) if i != desired]

    # Move the data to the GPU
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Convert the input data to the appropriate data type for the GPU
    train_X = train_X.to(device)
    train_y = train_y.to(device)

    ts_datasets = []
    if desired!=None:
        for noise_ind in noise_inds:
            TS_y, TS_X = head2head(train_y, train_X, desired=desired, noise=noise_ind)
            ts_datasets.append(torch.utils.data.TensorDataset(TS_X, TS_y))
    else:
        ts_datasets.append(torch.utils.data.TensorDataset(train_X, train_y))

    # Concatenate all datasets into one training set
    training_set = torch.utils.data.ConcatDataset(ts_datasets)
    
    return training_set

def QuickNN(training_set, n, num_epochs, batch_size, 
            l1_lambda = 0.0, stopEarly = 10, visualize=False):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Step 2: Define your neural network
    # Instantiate the neural network model
    inputs = training_set[0][0].shape[0]
    model = Net(input_size=inputs, num_classes=n,
                h1_size=n*2, h2_size=n*3, h3_size=n*4, dropout_prob=0
                ).to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=l1_lambda, amsgrad=True)

    # Define the learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                          factor=0.05, patience=5)
    
    # Step 3: Set up your training loop
    train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    best_loss = float('inf')  # initialize the best validation loss
    early_stop_counter = 0  # initialize the early stopping counter

    if stopEarly > 0:
        print("Early Stopping Initialized")
        # Create the validation set
        val_size = int(len(training_set) * 0.2) # Use 20% of the training set for validation
        val_set, train_set = random_split(training_set, [val_size, len(training_set) - val_size])
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)


    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            if len(inputs) == 0:
                continue
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()

            # Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if stopEarly > 0:
            # Define the learning rate scheduler
            lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                                factor=0.05, patience=5)
            # Evaluate the model on the validation set
            with torch.no_grad():
                val_loss = 0.0
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets).item()
            val_loss /= len(val_loader)

            # Check if the validation loss has improved
            if np.round(val_loss,5) < best_loss:
                best_loss = val_loss
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= stopEarly:  # if the validation loss hasn't improved for 10 epochs, stop training
                    print(f"Early stopping at: Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
                    break
        else:
            # Update the learning rate using the scheduler
            lr_scheduler.step(loss)


        if visualize:
            # Print the training loss and learning rate after every epoch
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        
    return model

def TestModel(test_X, test_y, model, visualize=True):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    test_X = test_X.to(device)
    test_y = test_y.to(device)

    # Evaluate the model on the test set
    with torch.no_grad():
        outputs = model(test_X)
        _, predicted = torch.max(outputs.data, 1)

    results = pd.DataFrame()
    for i in range(min(y), max(y)+1):
        cells = sum(test_y==i).item()
        test_y_i = test_y==i
        y_pred_i = predicted==i
        TP = sum((test_y_i==1) & (y_pred_i==1)).item()
        FP = sum((test_y_i==0) & (y_pred_i==1)).item()
        TN = sum((test_y_i==0) & (y_pred_i==0)).item()
        FN = sum((test_y_i==1) & (y_pred_i==0)).item()
        TPR = TP / np.where(TP+FN == 0, np.nan, TP+FN)
        TNR = TN / np.where(TN+FP == 0, np.nan, TN+FP)
        Prec = TP / np.where(TP+FP == 0, np.nan, TP+FP)
        Accuracy = (TP+TN) / np.where(TP+FP+FN+TN == 0, np.nan, TP+FP+FN+TN)

        res_i = {'Cluster' : le.inverse_transform([i])[0],
            'cells' : cells,
            'TP' : TP,
            'FP' : FP,
            'TN' : TN,
            'FN' : FN,
            'TPR' : TPR,
            'TNR' : TNR,
            'Prec' : Prec,
            'Accuracy' : Accuracy}
        
        res_i = pd.DataFrame([res_i])
        results = pd.concat([results,res_i], ignore_index=True)
        
    if visualize:
        display(results.sort_values(by=['Cluster']))

    return results

In [6]:
n = len(np.unique(y))
num_epochs = 100 # specify the number of epochs to train for
batch_size = 32 # specify the batch size for training

curriculum_dict = {}
for i in range(0,45):
    curriculum_dict[i] = {}
    training_set = DesiredGeneCurriculum(train_y, train_X, desired=i)
    model = QuickNN(training_set, n, num_epochs, batch_size, stopEarly=5, visualize=False)
    results = TestModel(test_X, test_y, model, visualize=True)
    curriculum_dict[i]['model'] = model
    curriculum_dict[i]['results'] = results

Early Stopping Initialized
Early stopping at: Epoch 65/100, Loss: 0.0000, Learning rate: 0.001000


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,489,6,5946,1,0.997959,0.998992,0.987879,0.998913
1,02_W3D1.2,502,495,4,5936,7,0.986056,0.999327,0.991984,0.998292
2,03_FminiON,321,315,1,6120,6,0.981308,0.999837,0.996835,0.998913
3,04_FminiOFF,354,351,2,6086,3,0.991525,0.999671,0.994334,0.999224
4,05_J-RGC,323,317,5,6114,6,0.981424,0.999183,0.984472,0.998292
5,06_W3B,277,270,9,6156,7,0.974729,0.998540,0.967742,0.997516
6,07_Novel,302,296,6,6134,6,0.980132,0.999023,0.980132,0.998137
7,08_Novel,244,239,3,6195,5,0.979508,0.999516,0.987603,0.998758
8,09_Tbr1_Novel,227,224,2,6213,3,0.986784,0.999678,0.991150,0.999224
9,10_Novel,240,237,4,6198,3,0.987500,0.999355,0.983402,0.998913


Early Stopping Initialized
Early stopping at: Epoch 65/100, Loss: 0.0000, Learning rate: 0.001000


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,486,3,5949,4,0.991837,0.999496,0.993865,0.998913
1,02_W3D1.2,502,497,12,5928,5,0.990040,0.997980,0.976424,0.997361
2,03_FminiON,321,314,1,6120,7,0.978193,0.999837,0.996825,0.998758
3,04_FminiOFF,354,349,2,6086,5,0.985876,0.999671,0.994302,0.998913
4,05_J-RGC,323,313,3,6116,10,0.969040,0.999510,0.990506,0.997982
5,06_W3B,277,273,6,6159,4,0.985560,0.999027,0.978495,0.998448
6,07_Novel,302,296,6,6134,6,0.980132,0.999023,0.980132,0.998137
7,08_Novel,244,238,2,6196,6,0.975410,0.999677,0.991667,0.998758
8,09_Tbr1_Novel,227,225,4,6211,2,0.991189,0.999356,0.982533,0.999069
9,10_Novel,240,238,4,6198,2,0.991667,0.999355,0.983471,0.999069


Early Stopping Initialized
Early stopping at: Epoch 83/100, Loss: 0.0000, Learning rate: 0.001000


...

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,488,7,5945,2,0.995918,0.998824,0.985859,0.998603
1,02_W3D1.2,502,496,11,5929,6,0.988048,0.998148,0.978304,0.997361
2,03_FminiON,321,311,1,6120,10,0.968847,0.999837,0.996795,0.998292
3,04_FminiOFF,354,350,2,6086,4,0.988701,0.999671,0.994318,0.999069
4,05_J-RGC,323,315,2,6117,8,0.975232,0.999673,0.993691,0.998448
5,06_W3B,277,271,10,6155,6,0.978339,0.998378,0.964413,0.997516
6,07_Novel,302,291,5,6135,11,0.963576,0.999186,0.983108,0.997516
7,08_Novel,244,227,3,6195,17,0.930328,0.999516,0.986957,0.996895
8,09_Tbr1_Novel,227,226,3,6212,1,0.995595,0.999517,0.986900,0.999379
9,10_Novel,240,236,1,6201,4,0.983333,0.999839,0.995781,0.999224


Early Stopping Initialized


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,487,6,5946,3,0.993878,0.998992,0.987830,0.998603
1,02_W3D1.2,502,495,10,5930,7,0.986056,0.998316,0.980198,0.997361
2,03_FminiON,321,317,6,6115,4,0.987539,0.999020,0.981424,0.998448
3,04_FminiOFF,354,346,4,6084,8,0.977401,0.999343,0.988571,0.998137
4,05_J-RGC,323,311,2,6117,12,0.962848,0.999673,0.993610,0.997827
5,06_W3B,277,265,6,6159,12,0.956679,0.999027,0.977860,0.997206
6,07_Novel,302,294,6,6134,8,0.973510,0.999023,0.980000,0.997827
7,08_Novel,244,232,3,6195,12,0.950820,0.999516,0.987234,0.997672
8,09_Tbr1_Novel,227,225,7,6208,2,0.991189,0.998874,0.969828,0.998603
9,10_Novel,240,238,1,6201,2,0.991667,0.999839,0.995816,0.999534


Early Stopping Initialized


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,488,9,5943,2,0.995918,0.998488,0.981891,0.998292
1,02_W3D1.2,502,491,5,5935,11,0.978088,0.999158,0.989919,0.997516
2,03_FminiON,321,315,1,6120,6,0.981308,0.999837,0.996835,0.998913
3,04_FminiOFF,354,347,3,6085,7,0.980226,0.999507,0.991429,0.998448
4,05_J-RGC,323,312,2,6117,11,0.965944,0.999673,0.993631,0.997982
5,06_W3B,277,274,10,6155,3,0.989170,0.998378,0.964789,0.997982
6,07_Novel,302,292,8,6132,10,0.966887,0.998697,0.973333,0.997206
7,08_Novel,244,232,5,6193,12,0.950820,0.999193,0.978903,0.997361
8,09_Tbr1_Novel,227,225,6,6209,2,0.991189,0.999035,0.974026,0.998758
9,10_Novel,240,237,4,6198,3,0.987500,0.999355,0.983402,0.998913


In [7]:
def compute_feature_importance(model, input_data, target_category):
    input_data.requires_grad = True # tell PyTorch to compute gradients with respect to the input
    model.zero_grad()
    output = model(input_data)
    # compute the negative log likelihood loss between the output and the target category
    loss = nn.functional.nll_loss(output, target_category) 
    # compute the gradients of the loss with respect to the input.
    loss.backward()
    # feature importance as the mean absolute value of the gradients over the batch dimension (i.e., over all input examples).
    feature_importance = input_data.grad.abs().mean(dim=0)
    return feature_importance.to('cpu')



# Convert train_X and test_X to PyTorch tensors on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_X = train_X.to(device)
test_X = test_X.to(device)

# Convert input data to a PyTorch tensor and move it to the GPU
input_data = torch.Tensor(test_X).to(device)

# Compute feature importance for each target category
for i in range(0, 45):
    mdl = curriculum_dict[i]['model'].to(device)
    target_category = torch.full((input_data.shape[0],), i, device=device, dtype=torch.long)
    curriculum_dict[i]['feature importance'] = compute_feature_importance(mdl, input_data, target_category)
    top_n_values, curriculum_dict[i][f'top {N} features'] = torch.topk(curriculum_dict[i]['feature importance'], N, largest=True)
    curriculum_dict[i][f'top {N} genes'] = list(df.columns[curriculum_dict[i][f'top {N} features']])
    print(curriculum_dict[i][f'top {N} genes'])


['Kcnip4', 'Pcdh7', 'Lxn', 'Col12a1', 'Isl2', 'RP23-407N2.2', 'Car8', 'Gabrg3', 'Prlr', 'Pou6f2']
['Prokr1', 'Maf', 'Zic1', 'Pcdh7', 'Kit', 'Nefh', 'Kcnk2', 'Car8', 'Gabrg3', 'Ano3']
['Kcnk2', 'Zic1', 'Prokr1', 'Isl1', 'Gabrg3', 'Prlr', 'Syndig1l', 'Nr4a2', 'Ptgds', 'Gngt1']
['Zic1', 'Kcnk2', 'Gngt1', 'Syndig1l', 'Fes', 'Tfap2d', 'Hap1', 'Bhlhe22', 'Gabrg3', 'Prokr1']
['Zic1', 'Kcnk2', 'Gngt1', 'Fes', 'Tfap2d', 'Gabrg3', 'Chrm2', 'Syndig1l', 'Pcdh20', 'Prokr1']
['Gabrg3', 'Kcnk2', 'Gngt1', 'Tfap2d', 'Chrm2', 'Wscd1', 'Syt6', 'Igfbp5', 'Tbx20', 'Calca']
['Kcnk2', 'Tfap2d', 'Gngt1', 'Syndig1l', 'Ebf3', 'Igfbp5', 'Pcdh20', 'Calca', 'Neurod2', 'Pou3f1']
['Kcnk2', 'Ebf3', 'Vit', 'Pcdh20', 'Gngt1', 'Syndig1l', 'Igfbp5', 'Tfap2d', 'Gabrg3', 'Calca']
['Ebf3', 'Igfbp5', 'Vit', 'Tfap2d', 'Syndig1l', 'Isl2', 'Gabrg3', 'Pcdh20', 'Meis2', 'Neurod2']
['Ebf3', 'Vit', 'Gabrg3', 'Pcdh20', 'Isl2', 'Syndig1l', 'Meis2', 'Tfap2d', 'Calca', 'Igfbp5']
['Pcdh20', 'Meis2', 'Syndig1l', 'Tfap2d', 'Isl2', 'Vit', 

In [8]:
# Load Tran Hypothesis
hypothesis = rdata['hypothesis']

g1 = hypothesis['Gene1'].copy()
g2 = hypothesis['Gene2'].copy()
g3 = hypothesis['Gene3'].copy()

tran_genes = np.concatenate([g1,g2,g3])
tran_genes = tran_genes[~pd.isna(tran_genes)]

tran_genes = np.where(tran_genes == '4833423E24Rik', 'four833423E24Rik', tran_genes)
tran_genes = np.unique(tran_genes)

tran_gene_indexes = [i for i, col in enumerate(df.columns) if any(gene in col for gene in tran_genes)]

def geneSelector(X, genes,device=device):
    X = X.to(device)
    # copy the values from X into X_zeros for the columns we want to keep
    X_sub = X[:, genes]
    return X_sub.to(device)

train_X_tran = geneSelector(train_X, tran_gene_indexes)
test_X_tran = geneSelector(test_X, tran_gene_indexes)


curriculum_dict['Tran'] = {}
training_set = DesiredGeneCurriculum(train_y, train_X_tran, desired=None)
model = QuickNN(training_set, n, num_epochs, batch_size, stopEarly=0, visualize=False)
results = TestModel(test_X_tran, test_y, model, visualize=True)
curriculum_dict['Tran']['model'] = model
curriculum_dict['Tran']['results'] = results

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,418,125,5827,72,0.853061,0.978999,0.769797,0.969419
1,02_W3D1.2,502,453,59,5881,49,0.902390,0.990067,0.884766,0.983235
2,03_FminiON,321,282,32,6089,39,0.878505,0.994772,0.898089,0.988979
3,04_FminiOFF,354,312,31,6057,42,0.881356,0.994908,0.909621,0.988668
4,05_J-RGC,323,262,72,6047,61,0.811146,0.988233,0.784431,0.979354
5,06_W3B,277,268,17,6148,9,0.967509,0.997242,0.940351,0.995964
6,07_Novel,302,277,34,6106,25,0.917219,0.994463,0.890675,0.990841
7,08_Novel,244,192,23,6175,52,0.786885,0.996289,0.893023,0.988358
8,09_Tbr1_Novel,227,196,41,6174,31,0.863436,0.993403,0.827004,0.988823
9,10_Novel,240,204,45,6157,36,0.850000,0.992744,0.819277,0.987426


In [9]:
top_N_gene_list = [ curriculum_dict[i][f'top {N} genes'] for i in range(0, 45)]

top_N_gene_list = np.unique(top_N_gene_list)

print(f' There are {len(top_N_gene_list)} unique genes found by the top {N} gradients from the curriculum models. They are:')
# x = [print(tg) for tg in top_N_gene_list]
print(top_N_gene_list)

common_genes = set(tran_genes).intersection(set(top_N_gene_list))
print(f'\nThere are {len(common_genes)} genes found by gradient that are in the Tran Proposal they are:')
x = [print(cg) for cg in common_genes]

 There are 47 unique genes found by the top 10 gradients from the curriculum models. They are:
['Ano3' 'Apela' 'Bhlhe22' 'Calca' 'Car8' 'Chrm2' 'Col12a1' 'Col25a1'
 'Ebf3' 'Fes' 'Gabrg3' 'Gngt1' 'Hap1' 'Igfbp5' 'Isl1' 'Isl2' 'Kcnip4'
 'Kcnk2' 'Kit' 'Lxn' 'Lypd1' 'Maf' 'Meis2' 'Mmp17' 'Necab2' 'Nefh'
 'Neurod2' 'Nr4a2' 'Pcdh11x' 'Pcdh20' 'Pcdh7' 'Penk' 'Pou3f1' 'Pou6f2'
 'Prlr' 'Prokr1' 'Ptgds' 'RP23-407N2.2' 'Slc17a7' 'Syndig1l' 'Syt6'
 'Tbx20' 'Tfap2d' 'Vit' 'Wscd1' 'Zic1' 'four833423E24Rik']

There are 20 genes found by gradient that are in the Tran Proposal they are:
Tbx20
Zic1
Prlr
Penk
Bhlhe22
Slc17a7
Mmp17
Prokr1
four833423E24Rik
Apela
Vit
Kit
Fes
Chrm2
Igfbp5
Col25a1
Neurod2
Pcdh20
Calca
Lypd1


In [10]:
hand_picked = ['Opn4', 'Spp1', 'Serpine2', 'Cdhr1', 'Adcyap1', 'Nmb', 'Bhlhe22', 'Fxyd6', 'Stxbp6', 'Coch']
hand_picked = ['Bhlhe22', 'Stxbp6', 'Cdhr1', 'Adcyap1']#, '', 'Nmb']
top_N_gene_list = np.append(top_N_gene_list, hand_picked)
gene_inds_topN = [i for i, col in enumerate(df.columns) if any(g in col for g in top_N_gene_list)]
train_X_topN = geneSelector(train_X, gene_inds_topN)
test_X_topN = geneSelector(test_X, gene_inds_topN)

training_set = DesiredGeneCurriculum(train_y, train_X_topN, desired=None)
model = QuickNN(training_set, n, num_epochs, batch_size, stopEarly=0, visualize=False)
results = TestModel(test_X_topN, test_y, model, visualize=True)
curriculum_dict[f'Model all top {N}'] = {}
curriculum_dict[f'Model all top {N}']['model'] = model

curriculum_dict[f'Model all top {N}']['results'] = results


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,479,17,5935,11,0.977551,0.997144,0.965726,0.995654
1,02_W3D1.2,502,482,37,5903,20,0.960159,0.993771,0.928709,0.991152
2,03_FminiON,321,294,20,6101,27,0.915888,0.996733,0.936306,0.992704
3,04_FminiOFF,354,319,44,6044,35,0.901130,0.992773,0.878788,0.987737
4,05_J-RGC,323,269,55,6064,54,0.832817,0.991012,0.830247,0.983080
5,06_W3B,277,273,10,6155,4,0.985560,0.998378,0.964664,0.997827
6,07_Novel,302,292,17,6123,10,0.966887,0.997231,0.944984,0.995809
7,08_Novel,244,209,30,6168,35,0.856557,0.995160,0.874477,0.989910
8,09_Tbr1_Novel,227,176,33,6182,51,0.775330,0.994690,0.842105,0.986961
9,10_Novel,240,233,7,6195,7,0.970833,0.998871,0.970833,0.997827


In [11]:
# Open a file for writing
with open('/home/sam/scRNAseq/RNAscope/RNAscope/curriculum_models_top10Genes.pkl', 'wb') as f:
    # Use pickle to dump the list to the file
    pickle.dump(curriculum_dict, f)

In [12]:
# gradient = curriculum_dict[f'Model all top {N}'][f'results']
# tran_hyp = curriculum_dict['Tran']['results']
tran_hyp = TestModel(test_X_tran, test_y, curriculum_dict['Tran']['model'], visualize=False)
gradient = TestModel(test_X_topN, test_y, curriculum_dict[f'Model all top {N}'][f'model'], visualize=False)
# gradient = TestModel(test_X, test_y, curriculum_dict['h2'][f'model'], visualize=False)


# Add a header indicating which dataframe the row comes from
gradient.columns = pd.Index([(f'NN Top {N} Gradients', col) for col in gradient.columns])
tran_hyp.columns = pd.Index([(f'Tran Hypothesis', col) for col in tran_hyp.columns])

# Concatenate the dataframes horizontally
results = pd.concat([gradient, tran_hyp], axis=1)

# Drop the 'Cluster' column from the 'Tran Hypothesis' dataframe
results = results.drop(('Tran Hypothesis', 'Cluster'), axis=1)

# Calculate and add the 'TPR Delta' and 'Prec Delta' columns to the 'results' dataframe
results[('Comparison', 'TPR Delta')] = np.where(results[(f'NN Top {N} Gradients', 'TPR')].isna() | results[('Tran Hypothesis', 'TPR')].isna(),
                                                np.nan,
                                                results[(f'NN Top {N} Gradients', 'TPR')] - results[('Tran Hypothesis', 'TPR')])

results[('Comparison', 'Prec Delta')] = np.where(results[(f'NN Top {N} Gradients', 'Prec')].isna() | results[('Tran Hypothesis', 'Prec')].isna(),
                                                 np.nan,
                                                 results[(f'NN Top {N} Gradients', 'Prec')] - results[('Tran Hypothesis', 'Prec')])

# Display the results dataframe
display(results)

# Display only the rows where 'TPR Delta' or 'Prec Delta' is negative
mask = (results[('Comparison', 'TPR Delta')] < 0) | (results[('Comparison', 'Prec Delta')] < 0)
sorted_results = results.loc[mask, :].sort_values(by=(f'NN Top {N} Gradients', 'TPR'))

display(sorted_results)

delta_tpr = results[('Comparison', 'TPR Delta')]
mean_delta_tpr = delta_tpr.mean()

print("Mean TPR Delta:", mean_delta_tpr)

delta_prec = results[('Comparison', 'Prec Delta')]
mean_delta_prec = delta_prec.mean()

print("Mean Prec Delta:", mean_delta_prec)


NN Top 10 Gradients                                                         \
               Cluster cells   TP  FP    TN  FN       TPR       TNR      Prec   
0            01_W3D1.1   490  479  17  5935  11  0.977551  0.997144  0.965726   
1            02_W3D1.2   502  482  37  5903  20  0.960159  0.993771  0.928709   
2           03_FminiON   321  294  20  6101  27  0.915888  0.996733  0.936306   
3          04_FminiOFF   354  319  44  6044  35  0.901130  0.992773  0.878788   
4             05_J-RGC   323  269  55  6064  54  0.832817  0.991012  0.830247   
5               06_W3B   277  273  10  6155   4  0.985560  0.998378  0.964664   
6             07_Novel   302  292  17  6123  10  0.966887  0.997231  0.944984   
7             08_Novel   244  209  30  6168  35  0.856557  0.995160  0.874477   
8        09_Tbr1_Novel   227  176  33  6182  51  0.775330  0.994690  0.842105   
9             10_Novel   240  233   7  6195   7  0.970833  0.998871  0.970833   
10            11_Novel   150  138  15  6277  12  0.920000  0.997616  0.901961   
11          12_ooDS_NT   176  168   9  6257   8  0.954545  0.998564  0.949153   
12            13_Novel   193  178  10  6239  15  0.922280  0.998400  0.946809   
13         14_ooDS_Cck   167  149  10  6265  18  0.892216  0.998406  0.937107   
14            15_Novel   161  154   2  6279   7  0.956522  0.999682  0.987179   
15          16_ooDS_DV   155  150   6  6281   5  0.967742  0.999046  0.961538   
16          17_Tbr1_S1   149  139  12  6281  10  0.932886  0.998093  0.920530   
17            18_Novel   163  145  28  6251  18  0.889571  0.995541  0.838150   
18            19_Novel   128  127   7  6307   1  0.992188  0.998891  0.947761   
19            20_Novel   124  114   9  6309  10  0.919355  0.998575  0.926829   
20          21_Tbr1_S2   115  103  19  6308  12  0.895652  0.996997  0.844262   
21               22_M5   126   91  29  6287  35  0.722222  0.995408  0.758333   
22             23_W3D2    97   93   4  6341   4  0.958763  0.999370  0.958763   
23            24_Novel   113  106   6  6323   7  0.938053  0.999052  0.946429   
24            25_Novel    88   81   5  6349   7  0.920455  0.999213  0.941860   
25            26_Novel    93   89   7  6342   4  0.956989  0.998897  0.927083   
26            27_Novel    93   80  13  6336  13  0.860215  0.997952  0.860215   
27         28_FmidiOFF   103   69  20  6319  34  0.669903  0.996845  0.775281   
28            29_Novel    82   73   9  6351   9  0.890244  0.998585  0.890244   
29            30_Novel    99   91   8  6335   8  0.919192  0.998739  0.919192   
30               31_M2    77   66  16  6349  11  0.857143  0.997486  0.804878   
31          32_F_Novel    67   57   9  6366  10  0.850746  0.998588  0.863636   
32               33_M1    59   36   7  6376  23  0.610169  0.998903  0.837209   
33            34_Novel    65   62   3  6374   3  0.953846  0.999530  0.953846   
34            35_Novel    52   49   7  6383   3  0.942308  0.998905  0.875000   
35            36_Novel    41   30   7  6394  11  0.731707  0.998906  0.810811   
36            37_Novel    40   38   5  6397   2  0.950000  0.999219  0.883721   
37          38_FmidiON    34   33   7  6401   1  0.970588  0.998908  0.825000   
38            39_Novel    45   38   4  6393   7  0.844444  0.999375  0.904762   
39            40_M1dup    21   16   9  6412   5  0.761905  0.998598  0.640000   
40         41_AlphaONT    20   18   7  6415   2  0.900000  0.998910  0.720000   
41        42_AlphaOFFS    23   20   0  6419   3  0.869565  1.000000  1.000000   
42         43_AlphaONS    26   18   5  6411   8  0.692308  0.999221  0.782609   
43            44_Novel     9    5   1  6432   4  0.555556  0.999845  0.833333   
44        45_AlphaOFFT     8    7   0  6434   1  0.875000  1.000000  1.000000   

              ... Tran Hypothesis                                     \
    Accuracy  ...              TP   FP    TN  FN       TPR       TNR   
0   0.995654  ...             418  125  5827  72  0

NN Top 10 Gradients                                                         \
               Cluster cells   TP  FP    TN  FN       TPR       TNR      Prec   
32               33_M1    59   36   7  6376  23  0.610169  0.998903  0.837209   
27         28_FmidiOFF   103   69  20  6319  34  0.669903  0.996845  0.775281   
42         43_AlphaONS    26   18   5  6411   8  0.692308  0.999221  0.782609   
21               22_M5   126   91  29  6287  35  0.722222  0.995408  0.758333   
35            36_Novel    41   30   7  6394  11  0.731707  0.998906  0.810811   
39            40_M1dup    21   16   9  6412   5  0.761905  0.998598  0.640000   
8        09_Tbr1_Novel   227  176  33  6182  51  0.775330  0.994690  0.842105   
38            39_Novel    45   38   4  6393   7  0.844444  0.999375  0.904762   
31          32_F_Novel    67   57   9  6366  10  0.850746  0.998588  0.863636   
7             08_Novel   244  209  30  6168  35  0.856557  0.995160  0.874477   
30               31_M2    77   66  16  6349  11  0.857143  0.997486  0.804878   
40         41_AlphaONT    20   18   7  6415   2  0.900000  0.998910  0.720000   
3          04_FminiOFF   354  319  44  6044  35  0.901130  0.992773  0.878788   
19            20_Novel   124  114   9  6309  10  0.919355  0.998575  0.926829   
10            11_Novel   150  138  15  6277  12  0.920000  0.997616  0.901961   
24            25_Novel    88   81   5  6349   7  0.920455  0.999213  0.941860   
37          38_FmidiON    34   33   7  6401   1  0.970588  0.998908  0.825000   

              ... Tran Hypothesis                                              \
    Accuracy  ...              TP  FP    TN  FN       TPR       TNR      Prec   
32  0.995343  ...              49   7  6376  10  0.830508  0.998903  0.875000   
27  0.991618  ...              88   9  6330  15  0.854369  0.998580  0.907216   
42  0.997982  ...              24   3  6413   2  0.923077  0.999532  0.888889   
21  0.990065  ...              98  21  6295  28  0.777778  0.996675  0.823529   
35  0.997206  ...              34   6  6395   7  0.829268  0.999063  0.850000   
39  0.997827  ...              16   3  6418   5  0.761905  0.999533  0.842105   
8   0.986961  ...             196  41  6174  31  0.863436  0.993403  0.827004   
38  0.998292  ...              33   2  6395  12  0.733333  0.999687  0.942857   
31  0.997051  ...              65   5  6370   2  0.970149  0.999216  0.928571   
7   0.989910  ...             192  23  6175  52  0.786885  0.996289  0.893023   
30  0.995809  ...              72  15  6350   5  0.935065  0.997643  0.827586   
40  0.998603  ...              18   5  6417   2  0.900000  0.999221  0.782609   
3   0.987737  ...             312  31  6057  42  0.881356  0.994908  0.909621   
19  0.997051  ...             104   8  6310  20  0.838710  0.998734  0.928571   
10  0.995809  ...             133  12  6280  17  0.886667  0.998093  0.917241   
24  0.998137  ...              83   9  6345   5  0.943182  0.998584  0.902174   
37  0.998758  ...              26   4  6404   8  0.764706  0.999376  0.866667   

             Comparison             
    Accuracy  TPR Delta Prec Delta  
32  0.997361  -0.220339  -0.037791  
27  0.996274  -0.184466  -0.131936  
42  0.999224  -0.230769  -0.106280  
21  0.992394  -0.055556  -0.065196  
35  0.997982  -0.097561  -0.039189  
39  0.998758   0.000000  -0.202105  
8   0.988823  -0.088106   0.015101  
38  0.997827   0.111111  -0.038095  
31  0.998913  -0.119403  -0.064935  
7   0.988358   0.069672  -0.018546  
30  0.996895  -0.077922  -0.022708  
40  0.998913   0.000000  -0.062609  
3   0.988668   0.019774  -0.030833  
19  0.995654   0.080645  -0.001742  
10  0.995498   0.033333  -0.015281  
24  0.997827  -0.022727   0.039687  
37  0.998137   0.205882  -0.041667  

[17 rows x 21 columns]

Mean TPR Delta: 0.05189783799206588
Mean Prec Delta: 0.03735092479903395


In [13]:
# Convert input data to a PyTorch tensor and move it to the GPU
input_data = torch.Tensor(test_X_topN).to(device)

# Compute feature importance for each target category
for i in range(0, 45):
    mdl = curriculum_dict[f'Model all top {N}']['model'].to(device)
    target_category = torch.full((input_data.shape[0],), i, device=device, dtype=torch.long)
    curriculum_dict[i]['feature importance'] = compute_feature_importance(mdl, input_data, target_category)
    top_n_values, curriculum_dict[i][f'top {N} features R2'] = torch.topk(curriculum_dict[i]['feature importance'], N, largest=True)
    curriculum_dict[i][f'top {N} genes Round2'] = list(df.columns[curriculum_dict[i][f'top {N} features R2']])
    print(curriculum_dict[i][f'top {N} genes Round2'])

top_N_gene_list = [ curriculum_dict[i][f'top {N} genes Round2'] for i in range(0, 45)]

top_N_gene_list = np.unique(top_N_gene_list)

print(f' There are {len(top_N_gene_list)} unique genes found by the top {N} gradients from the curriculum models. They are:')
# x = [print(tg) for tg in top_N_gene_list]
print(top_N_gene_list)

common_genes = set(tran_genes).intersection(set(top_N_gene_list))
print(f'\nThere are {len(common_genes)} genes found by gradient that are in the Tran Proposal they are:')
x = [print(cg) for cg in common_genes]

['Brinp1', 'Ano3', 'Cdk15', 'Apela', 'BC048546', 'Bex1', 'Car8', 'Ceacam10', 'Car2', 'Car3']
['Brinp1', 'BC048546', 'Cdk15', 'Ano3', 'Ceacam10', 'Apela', 'Car2', 'Bhlhe22', 'Atp2b4', 'Car3']
['BC048546', 'Ceacam10', 'Car2', 'Ano3', 'Brinp1', 'Bhlhe22', 'Atp2b4', 'Apela', 'Chrna3', 'Cdk15']
['Car2', 'Ceacam10', 'BC048546', 'Atp2b4', 'two510009E07Rik', 'Chrna3', 'Chrna6', 'Bhlhe22', 'Apela', 'Cdk15']
['Ceacam10', 'Car2', 'BC048546', 'Apela', 'Chrna3', 'Atp2b4', 'Chrm2', 'Brinp1', 'two510009E07Rik', 'Chrna6']
['Car2', 'Ceacam10', 'BC048546', 'Chrm2', 'Atp2b4', 'two510009E07Rik', 'Chrna6', 'Apela', 'Brinp1', 'Chrna3']
['Car2', 'Ceacam10', 'BC048546', 'two510009E07Rik', 'Chrna3', 'Atp2b4', 'Brinp1', 'Chrna6', 'Chrm2', 'Cdk15']
['Car2', 'Ceacam10', 'Brinp1', 'two510009E07Rik', 'BC048546', 'Alcam', 'Chrna3', 'Chrm2', 'Atp2b4', 'Chrna6']
['Brinp1', 'Ceacam10', 'two510009E07Rik', 'Car2', 'Chrna3', 'Alcam', 'Apela', 'Chrm2', 'Atp2b4', 'Bdnf']
['Brinp1', 'two510009E07Rik', 'Car2', 'Alcam', 'Ceaca

In [14]:
gene_inds_topN = [i for i, col in enumerate(df.columns) if any(g in col for g in top_N_gene_list)]
train_X_topN = geneSelector(train_X, gene_inds_topN)
test_X_topN = geneSelector(test_X, gene_inds_topN)

training_set = DesiredGeneCurriculum(train_y, train_X_topN, desired=None)
model = QuickNN(training_set, n, num_epochs, batch_size, stopEarly=0, visualize=False)
results = TestModel(test_X_topN, test_y, model, visualize=True)
curriculum_dict[f'Model all top {N} Round2'] = {}
curriculum_dict[f'Model all top {N} Round2']['model'] = model

curriculum_dict[f'Model all top {N} Round2']['results'] = results

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,490,326,216,5736,164,0.665306,0.963710,0.601476,0.941012
1,02_W3D1.2,502,387,179,5761,115,0.770916,0.969865,0.683746,0.954362
2,03_FminiON,321,261,127,5994,60,0.813084,0.979252,0.672680,0.970972
3,04_FminiOFF,354,244,190,5898,110,0.689266,0.968791,0.562212,0.953431
4,05_J-RGC,323,170,147,5972,153,0.526316,0.975976,0.536278,0.953431
5,06_W3B,277,115,149,6016,162,0.415162,0.975831,0.435606,0.951723
6,07_Novel,302,210,117,6023,92,0.695364,0.980945,0.642202,0.967557
7,08_Novel,244,117,109,6089,127,0.479508,0.982414,0.517699,0.963365
8,09_Tbr1_Novel,227,92,92,6123,135,0.405286,0.985197,0.500000,0.964762
9,10_Novel,240,211,41,6161,29,0.879167,0.993389,0.837302,0.989134


Mean TPR Delta: 0.06800616558678353
Mean Prec Delta: 0.05856365051393038
